In [1]:
import matplotlib as plt2 #need this for patches for shaded circles
import numpy as np
import matplotlib.pyplot as plt
import gizmo_analysis as gizmo 
import utilities as ut
import matplotlib.colors as colors
from matplotlib import rc #to use Latex math symbols like 'phi'
import astropy
from astropy.io import ascii
import matplotlib

import pdb
from importlib import reload
# from sl_utilities import distance_functions
#pdb.set_trace()  #<--in case need to troubleshoot
import pickle
import os
from fof_analysis import fof

In [3]:
#########################################################################
#########################################################################
#load particle data here as you normally do to get the star information
#cut on Rxy, |z| and age
#run fof
#sind, sxcm, sycm, szcm, smtot, sgrpid, sr90, sr50, srmax =fof.find(x[si],y[si],z[si], b=b_kpc, mass=mass[si], ncut=ncut_min)
#srcm = np.sqrt(sxcm**2. + sycm**2.)

simname = 'm12i_res7100_mhdcv'
simtype="fire2"
simdir='/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12i_r7100/'
#simdir = '/scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12i_r7100/mhdcv/1Myr/fire2/'
snapnumber = 600
part = gizmo.io.Read.read_snapshots(['star'],'index', snapnumber, simulation_name=simname, simulation_directory=simdir, assign_hosts_rotation=True, assign_hosts=True)  



# in utilities.simulation.Snapshot():
* reading:  scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12i_r7100/snapshot_times.txt

  using snapshot index = 600, redshift = 0.000


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12i_r7100/output/snapdir_600/snapshot_600.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 57060074 particles
    star      (id = 4): 13976485 particles
    blackhole (id = 5): 0 particles

* reading the following
  species: ['star']

* reading particles from:
    snapshot_600.0.hdf5
    snapshot_600.1.hdf5
    snapshot_600.2.hdf5
    snapshot_600.3.hdf5

* reading cosmological parameters from:  scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12i_r7100/initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties


# in gizmo_analysis.gizmo_track.

In [10]:
rxyz     = part['star'].prop('host.distance.total')
Rxy      = part['star'].prop('host.distance.principal.cylindrical')[:,0]
x        = part['star'].prop('host.distance.principal')[:,0]
y        = part['star'].prop('host.distance.principal')[:,1]
z        = part['star'].prop('host.distance.principal')[:,2] 
mass     = part['star'].prop('mass')
feh      = part['star'].prop('metallicity.fe')
ids      = part['star'].prop('id')
id_child = part['star'].prop('id.child')
age      = part['star'].prop('age')


min_age_value = 5.42
max_age_value = 5.46
#select young stars within the disk for star clusters
keep = np.where((age >= min_age_value) & (age <= max_age_value) & ((Rxy < 20) & (Rxy>2)) & (abs(z) < 1.5))
#to run cluster finding on
rxyz0     = rxyz[keep]
Rxy0      = Rxy[keep]
x0        = x[keep]
y0        = y[keep]
z0        = z[keep]
mass0     = mass[keep]
feh0      = feh[keep]
id0       = ids[keep]
id_child0 = id_child[keep]
age0      = age[keep]

In [11]:
# print(x0)
# print(mass0)
# print(y0)
# print(ncut)
# ncut           = 5 #4 star particles
# ind, xcm, ycm, zcm, mtot, grpid, r90, r50, rmax, feh_avg, age_avg =fof.find(x0,y0,z0,feh=feh0,age=age0,b=linking_length, mass=mass0, ncut=ncut)

In [12]:
linking_length = 0.004 #4 parsec
ncut           = 5 #4 star particles

ind, xcm, ycm, zcm, mtot, grpid, r90, r50, rmax =fof.find(x0,y0,z0,b=linking_length, mass=mass0, ncut=ncut)
ngroup = len(mtot)

export_cluster={}
for grp_index in range(ngroup):  #iterate over each group
    cluster={}
    ids_in_cluster = id0[ind[grp_index]]  #these are the star particle ids in each cluster
    id_children_in_cluster = id_child0[ind[grp_index]]
    age=age0[ind[grp_index]]
    nstar = len(ids_in_cluster)
    groupid=grpid[grp_index]
    print('------------------------------------------------------------------------------------------------------------------')
    print('grpid, nstar, xcm (kpc), ycm (kpc), zcm (kpc), mtot (msun), rmax (pc)')
    print('%s     %i     %.4f     %.4f    %.4f     %.2e     %.1f ' % (grpid[grp_index], nstar, xcm[grp_index], ycm[grp_index], zcm[grp_index], mtot[grp_index], 1000*rmax[grp_index]))
    print("age of stars in the cluster",age)
    print('ids')
    string = '[' 
    for i in ids_in_cluster:
        string = string + str(i) + ', '

    #get rid of last extra ,
    length = len(string)-2
    string = string[0:length] + ']'
    print(string)

    print('id children')
    string = '[' 
    for i in id_children_in_cluster:
        string = string + str(i) + ', '

    #get rid of last extra ,
    length = len(string)-2
    string = string[0:length] + ']'
    print(string)
    print("These are the ids printed",ids_in_cluster)
    #feh_in_cluster=feh[ids_in_cluster]
    cluster={"cluster_groupid":groupid,"no_of_star":nstar,"id":ids_in_cluster,"id_children":id_children_in_cluster,"xcm":xcm[grp_index],"ycm":ycm[grp_index],"zcm":zcm[grp_index],"mtot":mtot[grp_index],"r90":r90[grp_index],"r50":r50[grp_index],"rmax":rmax[grp_index],"x":x0[ind[grp_index]],"y":y0[ind[grp_index]],"z":z0[ind[grp_index]],"age":age0[ind[grp_index]]}
    export_cluster.update({groupid:cluster})



######################################################################
#######################################################################

print('------------------------------------------------------------------------------------------------------------------')



linking length  0.004
min number particles in group  5
10.088620901107788 seconds
there are no groups with this linking length


TypeError: object of type 'numpy.uint32' has no len()

#### 4-5 Gyr started at 8:11 PM ended at 9:06 PM - 4 clusters found
#### 5-6 Gyr started at 4:11 PM ended at 5:04 PM - 2 clusters found
#### 6-7 Gyr started at 5:06 PM ended at 5:44 PM - no clusters found
#### 7-8 Gyr started at 5:49 PM ended at 5:50 PM - no clusters found
#### 8-9 Gyr started at something to something - no clusters found
#### 9-11 Gyr started at 6:56 PM ended at 

In [8]:
# instead of re running FOF to access cluster info, I am just extracting the data from a pickle file.


clusters_datapath="./"

file_name= "fire2_clusters_m12i_res7100_snapshot_600_5.0>age>6.0.pkl"

with open(clusters_datapath+file_name, "rb") as input:
  import_cluster = pickle.load(input)



In [9]:
print(age_avg[0], "Gyr")
print(age_avg[1], "Gyr")
# print(age_avg[5]*1000)
# print(age_avg[7]*1000)


for i in range(0,2):
    age=age0[ind[i]]
    print(np.std(age)*1000, 'Myr')

NameError: name 'age_avg' is not defined

In [10]:
# s5_max = '40'   # Myr
# s5_min = '1'

# age_cut_Myr_min = float(s5_min) #Myr
# age_cut_Myr_max = float(s5_max) #Myr
# age_cut_Gyr_min = age_cut_Myr_min / 1000. #Gyr
# age_cut_Gyr_max = age_cut_Myr_max / 1000. #Gyr

# si = np.where(((age <= age_cut_Gyr_max) & (age >= age_cut_Gyr_min)) & (rxyz < 30) & (abs(z) < 10)) #use this for selecting stars of interest
# fig = plt.figure(figsize=(10, 10))
# fig.subplots_adjust(wspace=0.22, left=0.1, right=0.95,
#                     bottom=0.12, top=0.95)

# image1 = plt.scatter(x[si],y[si],s=4,c=age[si]*1000,cmap='Greys',vmin=age_cut_Gyr_min*1000,vmax=age_cut_Gyr_max*1000)

plt.scatter(xcm,zcm,s=120,c='r')


NameError: name 'xcm' is not defined

In [11]:
x vs z plots edge on top down 

SyntaxError: invalid syntax (1332271033.py, line 1)

In [12]:
def first_snap_finder(age_array):
    #read in snapshot_times file here (maybe need to test astropy instead)
    data_file = np.genfromtxt("../snapshot_times.txt",names=True,skip_header=5)
    min_age = np.min(age_array)
    lookback_time = data_file["lookbacktimeGyr"]
    snapshot_number = data_file["i"]
    index = np.where(min_age>lookback_time)
    snaps_keep = snapshot_number[index]
    return(snaps_keep)

#     pretend I have an array called look_back time (this is the second to last column in snapshot_times)
# comment above

In [13]:
snaps_to_itterate_over = first_snap_finder([5,6])
print(snaps_to_itterate_over)

[385. 386. 387. 388. 389. 390. 391. 392. 393. 394. 395. 396. 397. 398.
 399. 400. 401. 402. 403. 404. 405. 406. 407. 408. 409. 410. 411. 412.
 413. 414. 415. 416. 417. 418. 419. 420. 421. 422. 423. 424. 425. 426.
 427. 428. 429. 430. 431. 432. 433. 434. 435. 436. 437. 438. 439. 440.
 441. 442. 443. 444. 445. 446. 447. 448. 449. 450. 451. 452. 453. 454.
 455. 456. 457. 458. 459. 460. 461. 462. 463. 464. 465. 466. 467. 468.
 469. 470. 471. 472. 473. 474. 475. 476. 477. 478. 479. 480. 481. 482.
 483. 484. 485. 486. 487. 488. 489. 490. 491. 492. 493. 494. 495. 496.
 497. 498. 499. 500. 501. 502. 503. 504. 505. 506. 507. 508. 509. 510.
 511. 512. 513. 514. 515. 516. 517. 518. 519. 520. 521. 522. 523. 524.
 525. 526. 527. 528. 529. 530. 531. 532. 533. 534. 535. 536. 537. 538.
 539. 540. 541. 542. 543. 544. 545. 546. 547. 548. 549. 550. 551. 552.
 553. 554. 555. 556. 557. 558. 559. 560. 561. 562. 563. 564. 565. 566.
 567. 568. 569. 570. 571. 572. 573. 574. 575. 576. 577. 578. 579. 580.
 581. 

In [14]:
# trouble shoot this bit so you can read in the data file
data_file = np.genfromtxt("../snapshot_times.txt",names=True,skip_header=5)
data_file.dtype

dtype([('i', '<f8'), ('scalefactor', '<f8'), ('redshift', '<f8'), ('timeGyr', '<f8'), ('lookbacktimeGyr', '<f8'), ('timewidthMyr', '<f8')])

In [9]:
num_clusters = []

age_increments_array = 0.1 #Gyr
age_width = 0.1 #Gyr
# age_increments_array = 1.0 #Gyr
# age_width = 1.0 #Gyr

# min_age_array_Gyr = np.arange(0,1,age_increments_array)
# max_age_array_Gyr = min_age_array_Gyr + age_width
    
min_age_array_Gyr = np.arange(0.0,0.2,age_increments_array)
max_age_array_Gyr = min_age_array_Gyr + age_width




print(min_age_array_Gyr)
print(max_age_array_Gyr)


b_parsec =  4.0 #linking length in parsecs
b_kpc = b_parsec / 1000.
ncut_min = 5 #min number of particles per group


n = len(min_age_array_Gyr)
print(n)
listd = []

for k in range(n):
    temporary_name = simname+'.'+str(snaps[0])+".minage"+ str('%.2f' %min_age_array_Gyr[k])+"_"+"maxage"+"_"+str('%.2f' %max_age_array_Gyr[k])+"_b"+str(b_parsec)+"_nmin"+str(ncut_min)+".pkl"
    
    listd.append(temporary_name)

# print(listd)


for i in range(len(min_age_array_Gyr)):



    age_cut_Gyr_min = min_age_array_Gyr[i]
    age_cut_Gyr_max = max_age_array_Gyr[i]



    si = np.where(((age <= age_cut_Gyr_max) & (age >= age_cut_Gyr_min)) & (rxyz < 30) & (abs(z) < 10)) #use this for selecting stars of interest
    small_sid = sid[si]
    small_child = sid_child[si]
    small_generation = sid_generation[si]
    small_age = age[si]
    small_mass = mass[si]
    small_x = x[si]
    small_y = y[si]
    small_z = z[si]
    small_rxyz = rxyz[si]
    small_feh = feh[si]

    
    ind, xcm, ycm, zcm, mtot, grpid, r90, r50, rmax, feh_avg, age_avg =fof.find(x[si],y[si],z[si],feh=feh[si],age=age[si],b=b_kpc, mass=mass[si], ncut=ncut_min)
    cluster_dic={"id":small_sid,"child":small_child,"generation":small_generation,"age":small_age,"mass":small_mass,"x":small_x,"y":small_y,"z":small_z,
                   "rxyz":small_rxyz,"ind":ind,"xcm":xcm,"ycm":ycm,"zcm":zcm,"mtot":mtot,"grpid":grpid,
                  "r90":r90,"r50":r50,"rmax":rmax,"feh_avg":feh_avg,"age_avg":age_avg,"feh":feh}
    
    with open(listd[i], 'wb') as output:
        pickle.dump(cluster_dic, output) #access the data using tracked_data_all_clusters_each_snap[snapshot][cluster_id]["x_tracked"])

    num_clusters.append(len(mtot))

[0.  0.1]
[0.1 0.2]
2


NameError: name 'snaps' is not defined

In [33]:
# instead of re running FOF to access cluster info, I am just extracting the data from a pickle file.


clusters_datapath="./"

file_name= "m12i_res7100.600.minage4.35_maxage_4.40_b4.0_nmin5.pkl"

with open(clusters_datapath+file_name, "rb") as input:
  import_clusterdata = pickle.load(input)